# Establishing a Connection - Carol
## Role: Invitee

If you want to start at Part 1 of this tutorial series go [here](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%201%20-%20Self-Sovereign%20Identity.ipynb).

In this notebook we'll be responding to an invitation from Alice through our Agents. This notebook has the following phases:

<b>Begin with the [Alice notebook](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>


6. Instatiate the controller for our Agent
7. Set up a message listener running as a service on our controller
8. Paste the invitation from the Alice notebook into the invitation variable here
9. Accept the invitation

<b>Carry on at the Alice side</b>
    
14. Check if connection active


### 6. Instatiate the controller for our Agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"
API_KEY = "carol_api_123456789"


agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

### 7. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using PyPubSub. The default listeners are used to update state and print logs.


In [3]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


### 8. Paste the invitation from the Alice notebook into the invitation variable here;

In [4]:
#Paste in invitation from Alice agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '9dc3acab-2e44-4d71-8a83-829ae6c0f916', 'serviceEndpoint': 'http://192.168.65.3:8020', 'recipientKeys': ['5egngyfvZcJ5xzAUy37NUQMXCESTRYudVDX5uFsCssvF'], 'label': 'Alice'}

### 9. Accept the invitation, then move to Alice's notebook

In [5]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection Handler Called
Connection 4101eddc-784a-4424-b739-2e38089c54db in State invitation
Connection Handler Called
Connection 4101eddc-784a-4424-b739-2e38089c54db in State request
Connection {'invitation_key': '5egngyfvZcJ5xzAUy37NUQMXCESTRYudVDX5uFsCssvF', 'their_label': 'Alice', 'my_did': '7HMi9D6RdnmkhMFrF9MmJm', 'state': 'request', 'created_at': '2023-01-02 09:05:27.076502Z', 'accept': 'manual', 'request_id': '47d22354-a2df-44cc-a815-ea6f62f1be55', 'invitation_mode': 'once', 'connection_id': '4101eddc-784a-4424-b739-2e38089c54db', 'routing_state': 'none', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-02 09:05:27.145195Z', 'their_role': 'inviter'}
Connection Handler Called
Connection 4101eddc-784a-4424-b739-2e38089c54db in State response
Connection Handler Called
Connection 4101eddc-784a-4424-b739-2e38089c54db in State active


### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [6]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Alice AGENT CONNECTION
{'invitation_key': '5egngyfvZcJ5xzAUy37NUQMXCESTRYudVDX5uFsCssvF', 'their_label': 'Alice', 'my_did': '7HMi9D6RdnmkhMFrF9MmJm', 'state': 'active', 'created_at': '2023-01-02 09:05:27.076502Z', 'accept': 'manual', 'their_did': 'P2PDkJkvX65RVKzvgsGC4g', 'request_id': '47d22354-a2df-44cc-a815-ea6f62f1be55', 'invitation_mode': 'once', 'connection_id': '4101eddc-784a-4424-b739-2e38089c54db', 'routing_state': 'none', 'rfc23_state': 'completed', 'updated_at': '2023-01-02 09:05:47.599080Z', 'their_role': 'inviter'}
State: active


## Optional: Remove connection

In [23]:
remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)

NameError: name 'alice_id' is not defined

In [7]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'invitation_key': 'AuBeW7Laq6SQzUAM5JNdwv13wdXeVDzxD3myp7euN5xq', 'their_label': 'Bob', 'my_did': 'CXVXVERJAYXkEVaS3EB4fT', 'state': 'active', 'created_at': '2023-01-02 08:17:19.920676Z', 'accept': 'manual', 'their_did': 'LpmiP5Lpre6u2BF76nSALw', 'invitation_mode': 'once', 'connection_id': 'b6614717-d3ae-4ddf-b368-c445d2358a34', 'routing_state': 'none', 'rfc23_state': 'completed', 'updated_at': '2023-01-02 08:20:24.167014Z', 'their_role': 'invitee'}, {'invitation_key': '5NqLisZNrS2AbeBNCQHcF7hZx22Xd6aYx8vxadbdkqmJ', 'their_label': 'Bob', 'my_did': 'QWLH91NGnms8E57znbMPbX', 'state': 'active', 'created_at': '2023-01-02 08:51:39.261613Z', 'accept': 'manual', 'their_did': '3jAoJUdyYtxQa6AkTbaeAm', 'invitation_mode': 'once', 'connection_id': '1c65e6ce-773a-4d92-bbd8-a966b1531b27', 'routing_state': 'none', 'rfc23_state': 'completed', 'updated_at': '2023-01-02 08:52:22.390542Z', 'their_role': 'invitee'}, {'invitation_key': '8XT4vniErAT5A7SDEK2bvDwz2dSYyacfABjjTSiN1Qwp', 'their_l

## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [8]:
response = await agent_controller.terminate()
print(response)

None
